<a href="https://colab.research.google.com/github/paolala24/senales_sistemas/blob/main/2_SLIT/Convolucion_Discreta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolución discreta y SLITs

- A continuación se ilustra el proceso de convolución y respuesta al impulso para resolver sistemas lineales e invariantes en el tiempo (SLIT) discreto.

- Dada la respuesta al impulso en tiempo discreto $h[n]\in\mathbb{R},\mathbb{C}$ de un SLIT $\mathsf{H}\{\cdot\},$ la respuesta del sistema $y[n]\in\mathbb{R},\mathbb{C}$ ante la entrada discreta $x[n]\in\mathbb{R},\mathbb{C}$, se puede calcular mediante el operador convolución:

$$y[n]=x[n]*h[n] = \sum^\infty_{k=-\infty}x[k]h[n-k].$$

- En general, se cuenta con vectores de tamaño finito, y con sistemas causales, por ende, si $x[n]$ cuenta con $N$ elementos y $h[n]$ con $L$ elementos, la salida $y[n]$ tendra, en el mejor de los casos, $M = N + L - 1$ elementos distintos de 0 (ver [interpretación gráfica convolución](https://en.wikipedia.org/wiki/Convolution)).


**Ejemplo**

- Se presenta la simulación del cálculo de la salida de un SLIT en tiempo discreto utilizando la respuesta al impulso y la convolución discreta implementando punto a punto el proceso de suma y comparando los resultados con la librería [convolve](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html) de numpy.

In [1]:
%matplotlib inline
from ipywidgets import interact #crear graficos interactivos
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal #crear señales prototipo en tiempo discreto


In [2]:
#crear señales
#h = np.array([1,-2,0,1,-2]).reshape(-1,1)#np.r_[np.zeros((5,1)),np.ones((5,1)),np.zeros((5,1))] # se genera un pulso rectangular
hu = np.array([-1,6,-10,3,1,-10,2,5])
h = np.diff(hu).reshape(-1,1)
x = np.array([-15,5,-3,0,5,7,-1]).reshape(-1,1)#signal.triang(12).reshape(-1,1) # se genera una señal triangular y se trabaja con arreglos tipo columna
#funcion convolucion
def convgraf(h,x): #función general para pintar proceso de convolución de forma interactiva
   lx = len(x)
   lh = len(h)
   M = 2*lx+lh # se genera un vector de tamaño completo para visualizar todo el proceso paso a paso
   xm = np.zeros((M,M))
   hm = np.r_[np.zeros((lx,1)),h,np.zeros((lx,1))]
   ym = np.zeros((M,M))
   for i in range(M-lx+1):
     #print(i)
     xm[i:i+lx,i] = np.flip(x).reshape(-1)#filp realizar el proceso de reflejo sobre arreglos discretos
     ym[i,i] = xm[:,i].T.dot(hm) #multiplicacion entre vectores -> producto punto
   return xm, ym, hm

def plot_conv(k,xm,ym,hm): #dibujar arreglos
     plt.stem(xm[:,k],markerfmt='+',label='$x[k]$')
     plt.stem(hm,linefmt='g',markerfmt='.',label='$h[n-k]$')
     plt.stem(ym[:k,:].sum(axis=0),markerfmt='s',linefmt='r',label='$y[n]=\sum^{\infty}_{k=-\infty}x[k]h[n-k]$')
     plt.legend()
     plt.show()
     return

xm, ym, hm = convgraf(h,x)

/tmp/ipython-input-2-3146691479.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ym[i,i] = xm[:,i].T.dot(hm) #multiplicacion entre vectores -> producto punto


In [3]:
#@interact(m=range(xm.shape[0]-len(x)+1)) #define panel tipo selección en el gráfico interactivo
@interact(k=(0, xm.shape[0]-len(x), 1))
def show_frame(k=0):
    plot_conv(k,xm,ym,hm)
plt.show()

interactive(children=(IntSlider(value=0, description='k', max=14), Output()), _dom_classes=('widget-interact',…

**Nota**: Para resolver SLIT en tiempo discreto respecto a su respuesta escalón, puede utilizar el operador de derivada en tiempo discreto de numpy [diferencia discreta](https://numpy.org/doc/stable/reference/generated/numpy.diff.html) para estimar la respuesta al impulso $h[n]$ desde la respueta escalón $\mathsf{H}\{\epsilon[n]\}.$

- El paquete numpy tiene implementado la convolución discreta (Ver [numpy.convolve](https://numpy.org/doc/stable/reference/generated/numpy.convolve.html)).

In [4]:
y_ = np.convolve(x.reshape(-1),h.reshape(-1))
y_

array([-105,  275, -296,  143,  151, -260,   -6,   76,  -91,  -15,  110,
          9,   -3])